In [1]:
import numpy as np
import csv
from radiomics import featureextractor
import pydicom
import SimpleITK as sitk

In [2]:

#take the 60sth slice of the first patient for testing
csv_path = r"C:\Users\Diederik\OneDrive\Bureaublad\studie tn\Minor vakken Porto\IA CAD\GITHUB\LAB\Data files\CSV DATA FILES\LIDC-IDRI-0001\slice60.csv"


#read csv file an make it a list
with open(csv_path, 'r') as file:
    reader = csv.reader(file)
    data = list(reader)

#turning list into numpy array
slice_array = np.array(data, dtype=int)
slice_array = slice_array[1:513]


def remove_black(slice_array):
    # Find rows and columns that contain only -2048
    rows_to_keep = ~np.all(slice_array == -2048, axis=1)
    cols_to_keep = ~np.all(slice_array == -2048, axis=0)    
    # Use boolean indexing to select only rows and columns that are not all -2048
    new_array = slice_array[rows_to_keep][:, cols_to_keep]
    print("Original array:")
    print(slice_array)
    print("\nArray after removing rows and columns containing only -2048:")
    print(new_array)
    return new_array

#remove_black(slice_array

In [9]:
#DICOM FILE PATH
dicom_file_path = r"C:\Users\Diederik\OneDrive\Bureaublad\studie tn\Minor vakken Porto\IA CAD\GITHUB\LAB\DICOM SR images\manifest-1585232716547\LIDC-IDRI\LIDC-IDRI-0001\01-01-2000-NA-NA-30178\3000567.000000-Segmentation of Nodule 1 - Annotation Nodule 001-90548\1-1.dcm"
dicom_data = pydicom.dcmread(dicom_file_path)

In [13]:
# Create a SimpleITK image from the DICOM pixel array
sitk_image = sitk.GetImageFromArray(dicom_data.pixel_array)

# Extract PixelSpacing and SliceThickness from DICOM metadata
pixel_spacing = [float(dicom_data.PixelSpacing[0]), float(dicom_data.PixelSpacing[1]), float(dicom_data.SliceThickness)]
sitk_image.SetSpacing(pixel_spacing)

# Create a simple mask covering the entire image
mask = sitk.Image(dicom_data.pixel_array.shape, sitk.sitkUInt8)
mask.SetSpacing(pixel_spacing)

# Assuming that ImagePositionPatient gives the origin of the image
mask.SetOrigin([float(dicom_data.ImagePositionPatient[0]), float(dicom_data.ImagePositionPatient[1]), float(dicom_data.ImagePositionPatient[2])])

mask_array = sitk.GetArrayFromImage(mask)
mask_array[:, :] = 1  # Set all pixels to 1 to cover the entire image

# Initialize feature extractor
extractor = featureextractor.RadiomicsFeatureExtractor()

# Run feature extraction with the mask
feature_vector = extractor.execute(sitk_image, mask_array)

# Access and print features
print("First order features:")
for key, value in feature_vector.items():
    if key.startswith("original_"):
        print(f"{key}: {value}")

AttributeError: 'FileDataset' object has no attribute 'PixelSpacing'